# 인공신경망 기초 (2)

## 과제 내용 설명
1. 인공신경망의 오차 역전파 과정을 직접 필기하여 계산해주세요
2. 인공 신경망을 구현하는 실습파일을 완성해주세요

## 우수과제 선정 이유
개념을 하나 하나 정리하신 점이 인상 깊었습니다. 또한, 코드에 대한 해설과 함께 결과 해석까지 모두 해주셨습니다.

## Assignment 1) 오차 역전파 계산
![img1](https://imgur.com/1cgzZEa.jpg)

![img2](https://imgur.com/aAoAcit.jpg)


## Assignment 2) 인공 신경망 구현

In [1]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)] 
    #hidden layer*(1(bias term)+input수)->weight수 결정! 히든레이어로 갈때 가중치는 총 3개 발생
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    #히든레이어에서 output레이어로 넘어갈때 output수 2이므로 각각 2(히든레이어수+1)개씩 총 4개의 weight가 발생
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [2]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i] # 순전파 진행 
        #weight * inputs 곱한거를 return 값으로 받은 후
    return activation

def sigmoid(activation): #activation function으로 sigmoid 선택(영역을 영역으로 변환)
    return 1.0 / (1.0 + np.exp(-activation)) # 시그모이드 구현

#활성화 함수의 역할은! nn에서 입력받은 데이터를 다음층으로 출력할지를 결정

#하나의 노드가 1개이상의 노드와 연결되어있고
#데이터 입력을 받게 되는데 연결강도의 가중치의 합을 구하게 되고
#활성화 함수를 통해 weights 값의 크기에 따라 출력하게 되는 것!

#먼저 순전파 propagation을 진행한다
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)  #weight와 input값을 곱한 계산값을
            neuron['output'] = sigmoid(activation) # 나온 계산 값을 그대로 쓰나요? #아니요! 시그모이드 함수에 넣어줍니다
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠? #넵
        inputs = new_inputs
    return inputs #한번의 순전파를 거친 output이 다음 hiddenlayer의 input값이 된다

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

In [3]:
row = [1, 0, None]
forward_propagate(network,row)
for layer in network:
    print(layer)
    
#순전파 학습과정을 거친 후 weight는 바뀌지 않는다!
#가중치가 바뀌는 과정은 오류역전파 과정에서 일어남

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748}]


In [4]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output) 

[0.6629970129852887, 0.7253160725279748]


In [9]:
for i in reversed(range(len(network))):
    print(i)

1
0


In [5]:
def sigmoid_derivative(output):
    return output * (1.0 - output) # 시그모이드 미분

#오류역전파 진행합니다
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        #i = 0 일때 2번째로
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]: 
                    error += (neuron['weights'][j] * neuron['delta']) #앞에서 구한 delta값을 기반으로 error 구한다
                errors.append(error) 
        # i =1 일때 1번으로!
        else:
            for j in range(len(layer)): 
                neuron = layer[j]
                errors.append(expected[j] - neuron['output']) 
                # 역전파시 오차는 어떻게 설정했나요?
                #함수인자로 받은 예상값 expected와 앞서 순전파로 구한 output값의 차
        for j in range(len(layer)):
            neuron = layer[j] 
            neuron['delta'] =  errors[j] * sigmoid_derivative(neuron['output'])
            #델타값은 앞서 구한 오류 값 * 순전파 과정으로 구한 output을 시그모이드 미분한 함수에 넣은 값 으로 구한다
            # 시그모이드 함수를 사용한 역전파 

In [6]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [7]:
#역전파 과정을 토대로 가중치를 업데이트 시킨다
def weights_update(network, row, l_rate): 
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]] #앞서 구한 output을 input으로 
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j] #앞서 구한 델타값과 learning rate를 곱해서 가중치에 더하는 방식으로 업데이트시킨다
            neuron['weights'][-1] +=  l_rate * neuron['delta']  # 퍼셉트론 학습 규칙
            
#앞서 진행한 과정을 반복해서 error을 줄여나간다
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network,row) # 먼저 순전파진행 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]- outputs[i])**2 for i in range(len(expected))])
            # 예측값의 오차 합
            #sum error확인-> 학습을 진행하면서 error가 줄어드는지 확인해야하니까
            backward_propagate_error(network, expected) #그다음 역전파 진행
            weights_update(network, row, l_rate) #역전파 기반 가중치를 업데이트 시킨다
        #이과정을 지정한 epoch수만큼 반복
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [8]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [12]:
n_inputs = len(dataset[0]) - 1# 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
#데이터셋에 들어있는 개수는 끝에 output label값도 포함하고 있으므로 그건 빼고!
n_outputs = len(set([row[-1] for row in dataset])) # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? 
#데이터셋의 맨마지막 부분 label값이 몇가지인지 그걸 set집합으로 중복된거 제거해서 그 개수를 세어준다
#여기서는 0 1 두가지 이므로 2개
network = initialize_network(n_inputs, 2, n_outputs) #먼저 네트워크 초기 설정

for layer in network:
    print(layer) # 초기 네트워크 
#학습된 네트워크랑 초기 네트워크를 비교하기 위해 먼저 초기 네트워크 출력했습니다


train_network(network, dataset, 1.0, 1000, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.



# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시별로,hint : for문))
for layer in network:
    print(layer) # 학습된(최적화된) 네트워크

[{'weights': [0.762280082457942, 0.0021060533511106927, 0.4453871940548014]}, {'weights': [0.7215400323407826, 0.22876222127045265, 0.9452706955539223]}]
[{'weights': [0.9014274576114836, 0.030589983033553536, 0.0254458609934608]}, {'weights': [0.5414124727934966, 0.9391491627785106, 0.38120423768821243]}]
>epoch=0, lrate=1.000, error=6.153
>epoch=1, lrate=1.000, error=5.621
>epoch=2, lrate=1.000, error=5.606
>epoch=3, lrate=1.000, error=5.569
>epoch=4, lrate=1.000, error=5.467
>epoch=5, lrate=1.000, error=5.215
>epoch=6, lrate=1.000, error=4.842
>epoch=7, lrate=1.000, error=4.397
>epoch=8, lrate=1.000, error=3.908
>epoch=9, lrate=1.000, error=3.415
>epoch=10, lrate=1.000, error=2.937
>epoch=11, lrate=1.000, error=2.438
>epoch=12, lrate=1.000, error=2.051
>epoch=13, lrate=1.000, error=1.670
>epoch=14, lrate=1.000, error=1.401
>epoch=15, lrate=1.000, error=1.204
>epoch=16, lrate=1.000, error=1.046
>epoch=17, lrate=1.000, error=0.919
>epoch=18, lrate=1.000, error=0.815
>epoch=19, lrate=1

In [71]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) # 순전파 결과에서 어떤것이 최종 아웃풋이 되나요?
    #output값으로 나온 두 값중 더 큰 값의 index값을 최종 아웃풋으로 된다!

In [69]:
predict(network,row)

[0.19890425627850458, 0.7877146498611793]


1

In [72]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network,row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction)) #아주 잘 학습되었음!

실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
